Импорт данных

In [46]:
from corus import load_lenta
# from itertools import islice

path = 'data/lenta-ru-news.csv.gz'
records = load_lenta(path)
# Считать первые N строк:
data_iterator = (next(records) for x in range(100))  # Данный вариант позволяеть сохранить исходный <class 'generator'>
# data = list(islice(records, 0, 100))
# Файл не будет читаться полностью в отличие от варианта с file.readlines().

Обработка данных

In [47]:
import re
# import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# nltk.download('stopwords')

data_clear_list = []
for data in data_iterator:
    data_topic_lower = data.topic.lower()
    data_title_lower = data.title.lower()
    data_topic_punctuation = re.sub(r'[^\w\s]', '', data_topic_lower)  # Еще вариант составить список своих стоп слов
    data_title_punctuation = re.sub(r'[^\w\s]', '', data_title_lower)  # и проводить сравнение с ним
    data_title_stop_words = list(
        word for word in data_title_punctuation.split() if word not in stopwords.words('russian'))

    snowball = SnowballStemmer(language='russian')
    data_title_stem = list(map(snowball.stem, data_title_stop_words))

    # ...stop_words преобразован из списка в строку. str() не работает.
    # Учесть, что "".join не преобразует тип int()
    data_clear_list.append([data_topic_punctuation, " ".join(data_title_stop_words)])

#print(f'len data_clear list: {len(data_clear_list)}', *data_clear_list[:5], sep='\n')

In [48]:
import pandas

df = pandas.DataFrame(data_clear_list, columns=['topic', 'title'])
print(f'Размерность датасета: {df.shape}')
print(f'Data types:\n{df.dtypes}')
print(f'Просмотр среза данных:', df.head(), sep='\n')

Размерность датасета: (100, 2)
Data types:
topic    object
title    object
dtype: object
Просмотр среза данных:
         topic                                              title
0       россия  названы регионы россии самой высокой смертност...
1        спорт  австрия представила доказательств вины российс...
2  путешествия          обнаружено самое счастливое место планете
3          мир  сша раскрыли сумму расходов расследование росс...
4          мир  хакеры рассказали планах великобритании замини...


In [49]:
# Стастические сводка методом describe
print(df.describe())

       topic                                              title
count    100                                                100
unique    13                                                100
top      мир  названы регионы россии самой высокой смертност...
freq      19                                                  1


In [50]:
# Распределение  классов по атрибуту topic
print(df.groupby('topic').size())

topic
бывший ссср          11
дом                   4
из жизни              4
интернет и сми        5
культура              6
мир                  19
наука и техника       5
путешествия           2
россия               13
силовые структуры     2
спорт                16
ценности              2
экономика            11
dtype: int64


Присвоим метки для topic: Подход №1 - Найти и заменить https://clck.ru/34Nsh4

In [51]:
# topic_cat = {"topic": {"россия": 1,  "мир": 2, "спорт": 3, "путешествия": 4}}
# df = df.replace(topic_cat)

Присвоим метки для topic: Подход № 2 - Кодирование метки https://clck.ru/34Nsh4

In [52]:
df['topic'] = df['topic'].astype('category')
# df['topic_cat'] = df['topic'].cat.codes
df.insert(0, 'topic_cat', df['topic'].cat.codes)

In [53]:
print(f'Размерность датасета: {df.shape}')
print(f'Data types:\n{df.dtypes}')
print(f'Просмотр среза данных:', df.head(), sep='\n')

Размерность датасета: (100, 3)
Data types:
topic_cat        int8
topic        category
title          object
dtype: object
Просмотр среза данных:
   topic_cat        topic                                              title
0          8       россия  названы регионы россии самой высокой смертност...
1         10        спорт  австрия представила доказательств вины российс...
2          7  путешествия          обнаружено самое счастливое место планете
3          5          мир  сша раскрыли сумму расходов расследование росс...
4          5          мир  хакеры рассказали планах великобритании замини...


In [54]:
# Стастические сводка методом describe
print(df.describe())

       topic_cat
count  100.00000
mean     6.26000
std      3.78092
min      0.00000
25%      4.00000
50%      6.00000
75%     10.00000
max     12.00000


Формирование обучающей и тестовой выборки

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['title'], df['topic_cat'],
                                                    train_size=0.67,
                                                    random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(67,) (67,)
(33,) (33,)


Векторизация документов: BOW

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_bow = CountVectorizer()
X_train_bow_vector = vectorizer_bow.fit_transform(X_train)
X_test_bow_vector = vectorizer_bow.fit_transform(X_test)
print(X_train_bow_vector.shape)

(67, 365)


Выбор модели классификатора и её обучение

In [60]:
from sklearn import svm

model = svm.SVC(kernel='poly', degree=2)
model.fit(X_train_bow_vector, y_train)

SVC(degree=2, kernel='poly')

In [62]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X_train_bow_vector, y_train, scoring='accuracy')

/home/workpc/pyProjects/classifier/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


array([0.21428571, 0.21428571, 0.23076923, 0.30769231, 0.15384615])